# Uber Request Data Analysis

**Importing Libraries**

In [1]:
import pandas as pd
import numpy as np

**Importing Data**

In [2]:
data = pd.read_csv('Uber Request Data.csv')

In [3]:
data.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp
0,619,Airport,1.0,Trip Completed,11/7/2016 11:51,11/7/2016 13:00
1,867,Airport,1.0,Trip Completed,11/7/2016 17:57,11/7/2016 18:47
2,1807,City,1.0,Trip Completed,12/7/2016 9:17,12/7/2016 9:58
3,2532,Airport,1.0,Trip Completed,12/7/2016 21:08,12/7/2016 22:03
4,3112,City,1.0,Trip Completed,13-07-2016 08:33:16,13-07-2016 09:25:47


**Cheching Data**

In [4]:
data.shape

(6745, 6)

In [5]:
data.columns

Index(['Request id', 'Pickup point', 'Driver id', 'Status',
       'Request timestamp', 'Drop timestamp'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6745 entries, 0 to 6744
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Request id         6745 non-null   int64  
 1   Pickup point       6745 non-null   object 
 2   Driver id          4095 non-null   float64
 3   Status             6745 non-null   object 
 4   Request timestamp  6745 non-null   object 
 5   Drop timestamp     2831 non-null   object 
dtypes: float64(1), int64(1), object(4)
memory usage: 316.3+ KB


**Chechking Null Values**

In [7]:
data.isnull().sum()

Request id              0
Pickup point            0
Driver id            2650
Status                  0
Request timestamp       0
Drop timestamp       3914
dtype: int64

**Fixing Data Types**

In [8]:
data['Request timestamp'] = data['Request timestamp'].astype('datetime64')
data['Drop timestamp'] = data['Drop timestamp'].astype('datetime64')

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6745 entries, 0 to 6744
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Request id         6745 non-null   int64         
 1   Pickup point       6745 non-null   object        
 2   Driver id          4095 non-null   float64       
 3   Status             6745 non-null   object        
 4   Request timestamp  6745 non-null   datetime64[ns]
 5   Drop timestamp     2831 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(2)
memory usage: 316.3+ KB


In [10]:
data[np.isnan(data['Driver id'])].head(100)

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp
4095,1362,City,NaN,No Cars Available,2016-11-07 00:02:00,NaT
4096,1364,City,NaN,No Cars Available,2016-11-07 00:06:00,NaT
4097,1366,City,NaN,No Cars Available,2016-11-07 00:09:00,NaT
4098,2,Airport,NaN,No Cars Available,2016-11-07 00:23:00,NaT
4099,7,Airport,NaN,No Cars Available,2016-11-07 00:30:00,NaT
...,...,...,...,...,...,...
4190,337,City,NaN,No Cars Available,2016-11-07 07:50:00,NaT
4191,342,City,NaN,No Cars Available,2016-11-07 07:51:00,NaT
4192,349,City,NaN,No Cars Available,2016-11-07 07:51:00,NaT
4193,344,City,NaN,No Cars Available,2016-11-07 07:52:00,NaT


**We are geting Driver Id as NAN when the STATUS is 'NO cars Available' it means for car being unavailable no driver is assigned**

**We are Replacing NAN values in Driver id by Zero**

In [11]:
data['Driver id'] = data['Driver id'].fillna(0)

In [12]:
data[np.isnan(data['Driver id'])]

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp


We have converted all nan values in driver id to 0

**Total number of Request Received for two listed Pickup Points**

1. Airport
2. City

In [13]:
data.groupby('Pickup point')['Request id'].count()

Pickup point
Airport    3238
City       3507
Name: Request id, dtype: int64

Uber is receiving more number of requests for City Locations

In [14]:
data['Status'].unique()

array(['Trip Completed', 'Cancelled', 'No Cars Available'], dtype=object)

Each request placed can have three unique status : Completed , No car Available  and Cancelled

**Let's check number of Request categorized in each status**

In [39]:
data.groupby('Status')['Request id'].count().sort_values()

Status
Cancelled            1264
No Cars Available    2650
Trip Completed       2831
Name: Request id, dtype: int64

Maximum requests are succesfully completed

**We are adding new column which will contain the total time difference between request and drop**

In [16]:
data['Time Diff'] = data['Drop timestamp'] - data['Request timestamp']

In [17]:
data.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Time Diff
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,0 days 01:09:00
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,0 days 00:50:00
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,0 days 00:41:00
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,0 days 00:55:00
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,0 days 00:52:31


**We will be now converted time taken into number of minuted required**

In [18]:
data['Minutes Taken'] = data['Time Diff'].apply(lambda x:x.seconds/60)

In [19]:
data.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Time Diff,Minutes Taken
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,0 days 01:09:00,69.000000
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,0 days 00:50:00,50.000000
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,0 days 00:41:00,41.000000
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,0 days 00:55:00,55.000000
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,0 days 00:52:31,52.516667


In [20]:
n_data = data.drop(['Time Diff'], axis=1)
n_data

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Minutes Taken
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,69.000000
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,50.000000
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,41.000000
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,55.000000
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,52.516667
...,...,...,...,...,...,...,...
6740,6745,City,0.0,No Cars Available,2016-07-15 23:49:03,NaT,NaN
6741,6752,Airport,0.0,No Cars Available,2016-07-15 23:50:05,NaT,NaN
6742,6751,City,0.0,No Cars Available,2016-07-15 23:52:06,NaT,NaN
6743,6754,City,0.0,No Cars Available,2016-07-15 23:54:39,NaT,NaN


**We will create new column which will give us that in whichc hour the request was done**

In [21]:
data['Req Hours']  = data['Request timestamp'].apply(lambda x:x.hour)

In [22]:
data.head(5)

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Time Diff,Minutes Taken,Req Hours
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,0 days 01:09:00,69.000000,11
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,0 days 00:50:00,50.000000,17
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,0 days 00:41:00,41.000000,9
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,0 days 00:55:00,55.000000,21
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,0 days 00:52:31,52.516667,8


**We will now categorize our Hours in various sections like EARLY MORNING, MORNING,etc**

In [23]:
def getTimeOfDay(x):
    if x>=5 and x<8:
        tod="Early Morning"
    elif x>=8 and x<12:
        tod="Morning"
    elif x>=12 and x<16:
        tod="Afternoon"
    elif x>=16 and x<19:
        tod="Evening"
    elif x>=19 and x<23:
        tod="Night"
    else:
        tod="Late Night"
    return tod

In [24]:
data['Time of the Day'] = data['Req Hours'].apply(getTimeOfDay)

In [25]:
data.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Time Diff,Minutes Taken,Req Hours,Time of the Day
0,619,Airport,1.0,Trip Completed,2016-11-07 11:51:00,2016-11-07 13:00:00,0 days 01:09:00,69.000000,11,Morning
1,867,Airport,1.0,Trip Completed,2016-11-07 17:57:00,2016-11-07 18:47:00,0 days 00:50:00,50.000000,17,Evening
2,1807,City,1.0,Trip Completed,2016-12-07 09:17:00,2016-12-07 09:58:00,0 days 00:41:00,41.000000,9,Morning
3,2532,Airport,1.0,Trip Completed,2016-12-07 21:08:00,2016-12-07 22:03:00,0 days 00:55:00,55.000000,21,Night
4,3112,City,1.0,Trip Completed,2016-07-13 08:33:16,2016-07-13 09:25:47,0 days 00:52:31,52.516667,8,Morning


**Let's Find number of requests Received for each pickup point in various slots of the day**

In [40]:
data.groupby(['Time of the Day', 'Pickup point'])['Request id'].count().sort_values()

Time of the Day  Pickup point
Early Morning    Airport          264
Afternoon        Airport          278
Morning          Airport          301
Evening          City             313
Late Night       Airport          351
Afternoon        City             373
Late Night       City             421
Night            City             448
Evening          Airport          774
Morning          City             967
Early Morning    City             985
Night            Airport         1270
Name: Request id, dtype: int64

As we can see in table above that maximum reuests were for Airport Pickup Point and were received in Night

**Number of Requests and there Status for various Pickup Points**

In [41]:
data.groupby(['Status', 'Pickup point'])['Request id'].count()

Status             Pickup point
Cancelled          Airport          198
                   City            1066
No Cars Available  Airport         1713
                   City             937
Trip Completed     Airport         1327
                   City            1504
Name: Request id, dtype: int64

*Maximum requests cancelled were placed for City, Requests for which car was not available were from Airport and Most succesfull Trips were for City*

**Status of Trips in different TimeSlots**

In [28]:
supply_demand_timeslots=pd.pivot_table(data, 'Request id', ['Time of the Day'], ['Status'],aggfunc='count')
supply_demand_timeslots

Status,Cancelled,No Cars Available,Trip Completed
Time of the Day,,,
Afternoon,69,182,400
Early Morning,490,233,526
Evening,81,600,406
Late Night,75,380,317
Morning,430,279,559
Night,119,976,623


**Mean Time required for each Time slot**

In [29]:
Avgeragetime_timeslots=pd.pivot_table(data, 'Minutes Taken', ['Time of the Day'], ['Status'],aggfunc='mean')
Avgeragetime_timeslots

Status,Trip Completed
Time of the Day,
Afternoon,52.122375
Early Morning,52.703010
Evening,51.533662
Late Night,52.989905
Morning,52.487955
Night,52.570412


**Created new DataFrame which contains only cancelled and incomplete requests**

In [30]:
incomplete = ['Cancelled', 'No Cars Available']
uber_incomplete_trip = data[data['Status'].isin(incomplete)]

In [31]:
uber_incomplete_trip.head()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Time Diff,Minutes Taken,Req Hours,Time of the Day
2831,2905,City,1.0,Cancelled,2016-07-13 06:08:41,NaT,NaT,NaN,6,Early Morning
2832,4805,City,1.0,Cancelled,2016-07-14 17:07:58,NaT,NaT,NaN,17,Evening
2833,5202,Airport,1.0,Cancelled,2016-07-14 20:51:37,NaT,NaT,NaN,20,Night
2834,5927,City,1.0,Cancelled,2016-07-15 10:12:40,NaT,NaT,NaN,10,Morning
2835,2347,Airport,2.0,Cancelled,2016-12-07 19:14:00,NaT,NaT,NaN,19,Night


In [32]:
uber_incomplete_trip.groupby('Pickup point')['Request id'].count()

Pickup point
Airport    1911
City       2003
Name: Request id, dtype: int64

*Max incomplete requests were for City*

In [33]:
uber_incomplete_trip.groupby(['Pickup point', 'Time of the Day'])['Request id'].count()

Pickup point  Time of the Day
Airport       Afternoon           91
              Early Morning       23
              Evening            581
              Late Night         186
              Morning             58
              Night              972
City          Afternoon          160
              Early Morning      700
              Evening            100
              Late Night         269
              Morning            651
              Night              123
Name: Request id, dtype: int64

In [34]:
uber_incomplete_trip.groupby(['Time of the Day', 'Status'])['Request id'].count()

Time of the Day  Status           
Afternoon        Cancelled             69
                 No Cars Available    182
Early Morning    Cancelled            490
                 No Cars Available    233
Evening          Cancelled             81
                 No Cars Available    600
Late Night       Cancelled             75
                 No Cars Available    380
Morning          Cancelled            430
                 No Cars Available    279
Night            Cancelled            119
                 No Cars Available    976
Name: Request id, dtype: int64

**New DataFrame that contains conly completed Trip Request data**

In [35]:
uber_complete_trip =  data[data['Status']== 'Trip Completed']

In [36]:
uber_complete_trip.sample()

,Request id,Pickup point,Driver id,Status,Request timestamp,Drop timestamp,Time Diff,Minutes Taken,Req Hours,Time of the Day
1058,469,Airport,110.0,Trip Completed,2016-11-07 09:24:00,2016-11-07 10:04:00,0 days 00:40:00,40.0,9,Morning


In [37]:
uber_complete_trip.groupby(['Pickup point', 'Time of the Day'])['Minutes Taken'].mean()

Pickup point  Time of the Day
Airport       Afternoon          51.507308
              Early Morning      51.611618
              Evening            52.600604
              Late Night         52.443535
              Morning            52.225034
              Night              52.867226
City          Afternoon          52.662363
              Early Morning      53.625906
              Evening            50.566901
              Late Night         53.583004
              Morning            52.690137
              Night              52.298256
Name: Minutes Taken, dtype: float64

In [42]:
uber_complete_trip.groupby(['Pickup point', 'Time of the Day'])['Request id'].count()

Pickup point  Time of the Day
Airport       Afternoon          187
              Early Morning      241
              Evening            193
              Late Night         165
              Morning            243
              Night              298
City          Afternoon          213
              Early Morning      285
              Evening            213
              Late Night         152
              Morning            316
              Night              325
Name: Request id, dtype: int64